In [2]:
import hana_ml
print(f"SAP HANA Client for Python: {hana_ml.__version__}")

SAP HANA Client for Python: 2.23.25010300


In [2]:
## HANA Trial
myhost='62f61f95-95b5-4ec0-a551-90d890eabbdf.hana.trial-us10.hanacloud.ondemand.com'
myport=443
myuser='DBADMIN'

In [3]:
## HANA Trial
myhost='b7c3ff95-9e0c-480d-9022-bfaaa268f780.hna0.prod-us10.hanacloud.ondemand.com'
myport=443
myuser='DBADMIN_VR'

In [4]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

Connected to SAP HANA db version 4.00.000.00.1736264738 (fa/CE2024.28) 
at b7c3ff95-9e0c-480d-9022-bfaaa268f780.hna0.prod-us10.hanacloud.ondemand.com:443 as user DBADMIN_VR


# Load Colors Data

## Generate colors

In [33]:
import pandas as pd
import webcolors

specs=[webcolors.HTML4]


In [34]:
for spec in specs:
    # Display the `spec`
    print(spec)
    
    # Generate the list of color names and their RGB values
    colors_data = [
        (wc, *webcolors.name_to_rgb(wc)) 
        for wc in webcolors.names(spec=spec)
    ]

    # Create a DataFrame from the list
    df = pd.DataFrame(colors_data, columns=["Color_Name", "Red", "Green", "Blue"])

    hdf_colors=hdf.create_dataframe_from_pandas(
        connection_context=myconn,
        table_name=f'COLORS_{spec}',
        pandas_df=df,
        force=True  # Overwrites the table if it already exists
    )


html4


100%|██████████| 1/1 [00:00<00:00, 102.44it/s]


# Choose tables to experiment with

In [35]:
spec = specs[0]
hdf_colors=myconn.table(f'COLORS_{spec}')

In [36]:
hdf_colors.tail(10).collect()

,Color_Name,Red,Green,Blue
0,lime,0,255,0
1,maroon,128,0,0
2,navy,0,0,128
3,olive,128,128,0
4,purple,128,0,128
5,red,255,0,0
6,silver,192,192,192
7,teal,0,128,128
8,white,255,255,255
9,yellow,255,255,0


In [37]:
(myconn
    .get_tables()
)

,TABLE_NAME
0,COLORS_html4
1,COLORS_css3


In [38]:
df_colors=(myconn
    .sql(f'''
    WITH "SINGLE_COLOR" AS (SELECT TO_REAL_VECTOR(ARRAY("Red","Green","Blue")) AS "Ref_Color" FROM "COLORS_{spec}" WHERE "Color_Name"='white')
    SELECT TOP 1000
        "Color_Name",
        "Red",
        "Green",
        "Blue",
        ROUND(L2DISTANCE(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "L2Dist",
        ROUND(COSINE_SIMILARITY(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "CosSim"
    FROM "COLORS_{spec}", "SINGLE_COLOR"
    WHERE "Color_Name" <> 'black'
    ORDER BY "L2Dist", "CosSim" DESC ;
    ''')
    .collect()
)

In [39]:

def color_row(row):
    color_name = row["Color_Name"]
    # Add cell background color based on the color name
    return [f"background-color: {color_name}" for _ in row]

In [40]:
df_colors.style.apply(color_row, axis=1)

,Color_Name,Red,Green,Blue,L2Dist,CosSim
0,white,255,255,255,0.000000,1.000000
1,silver,192,192,192,109.120000,1.000000
2,gray,128,128,128,219.970000,1.000000
3,aqua,0,255,255,255.000000,0.820000
4,fuchsia,255,0,255,255.000000,0.820000
5,yellow,255,255,0,255.000000,0.820000
6,olive,128,128,0,311.900000,0.820000
7,purple,128,0,128,311.900000,0.820000
8,teal,0,128,128,311.900000,0.820000
9,blue,0,0,255,360.620000,0.580000
